In [236]:
import numpy as np;
import pandas as pd;
from scipy.optimize import fmin_bfgs;


        
        
# trainingDf= pd.read_csv("C:/Dinesh/kaggle/titanic/train.csv", index_col=0, parse_dates=True, nrows = 20);
trainingDf= pd.read_csv("D:/datasets/titanic-kaggle/train.csv", index_col=0, parse_dates=True, nrows = 20);

listRowsToColumns=["Sex", "Embarked"];
listRowsToBoolean=["Cabin"];
listNonFeatures = ["Name", "Ticket"];

def generateBooleanColumns(df, columns):
    for column in columns:
            df[column] = -1 * df[column].isnull() + 1
    return df;

def generateValueColumns(df, columns):
    for column in columns:
        df = pd.concat([df.drop(column, axis=1), df[column].str.get_dummies()], axis = 1);
    return df;

def removeNonFeatures(df, columns):
    for column in columns:
        df = df.drop(column, axis = 1);
    return df;
    
def dataTransform(df, listRowstoColumns, listRowsToBoolean, listNonFeatures):
    df1 = generateBooleanColumns(df, listRowsToBoolean);
    df2 = removeNonFeatures(df, listNonFeatures);
    return generateValueColumns(df2, listRowstoColumns);


def costFunction(theta, X, y, lam):
  
    theta = np.transpose(np.asmatrix(theta));
    
    m = X.shape[0];
    n = X.shape[1];
    print(
        "in cost function",
        X.shape,
        theta.shape
    );
    J = (
            (
                1/m
            ) 
            * np.sum( 
                        
                            (-1*y) 
                            * np.log(
                                    sigmoid(X*theta)
                            )
                        - 
                            np.subtract(1, y) 
                            * np.log(
                                1-sigmoid(X*theta)
                            )
            )  
    ) + \
        ( (lam/m) * sum(np.square(np.vstack([0, theta[1:]]))) )
    grad = ( (1/m) * (np.transpose(X) * (sigmoid(X*theta) - y)) ) + \
        ( (lam/m) * np.vstack([0, theta[1:]]) );
    return J[0][0]#, grad;
def gradFunction(theta, X, y, lam):
    m = X.shape[0];
    n = X.shape[1];
    theta = np.ndarray.flatten(np.asmatrix(theta));
#     theta = reshape(theta,(len(theta),1))
#     print(
#         "in grad function",
#         X.shape
#         , theta.shape
#     );
    
    grad = ( (1/m) * (
#             np.transpose(X) * 
        (
            sigmoid(X*theta) 
#             - y
        )
    ) ) + \
        ( (lam/m) * np.vstack([0, theta[1:]]) );
    return grad;
    

def sigmoid(z):
    return 1 / (1 + np.exp(-1 * z));
    
def logisticRegression(X, theta):
    return round(sigmoid(X*theta));

trainingDf = dataTransform(trainingDf, listRowsToColumns, listRowsToBoolean, listNonFeatures);   
trainingDf = trainingDf.fillna(trainingDf.mean())
X = trainingDf.ix[:, trainingDf.columns != 'Survived'].values;
y = trainingDf["Survived"].values

# predictions = sigmoid(X*theta);
X = np.matrix('1 1');
# print(X.shape)
theta = np.matrix('1;1');
# print(theta.shape)
lam = 0;
y = np.matrix('3');
# print(y.shape)
# print(sigmoid(X * theta))

#J, grad = costFunction(theta, X, y, 1);
print (
   # grad.flatten().tolist()[0],
    #J,
  #  grad
#     theta
);
theta = fmin_bfgs(costFunction, theta, maxiter=10, args=(X, y, 1), fprime=gradFunction)
theta


ValueError: shapes (1,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

In [295]:
class LogisticRegression:
    
    def __init__(self, X, y, lam):
        self.X = X;
        self.y = y;
        self.lam = lam;
        self.m = X.shape[0];
        self.n = X.shape[1];
        self.X = self.addIntercept(self.X); # to be checked for documentation
        self.theta = self.initializeTheta();
        
    def initializeTheta(self):
        return np.zeros((self.n + 1, 1));
    
    @staticmethod
    def addIntercept(X):
        m = X.shape[0];
        return np.hstack((np.ones((m,1)), X));
        
    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-1 * z));

    def buildModel(self):
        fminOutput = fmin_bfgs(
                                            self.costFunction,
                                            self.theta,
                                            self.gradFunction,
                                            disp=True,
                                            maxiter=400,
                                            full_output = True,
                                            retall=True
                                        );
        opTheta = fminOutput[0];
        opTheta = opTheta.reshape((self.n + 1, 1));
        return opTheta, fminOutput
        
    
        
    def costFunction(self, theta):
        
        theta = np.reshape(theta,(self.n + 1, 1));
        
        z = self.X * theta;
        h = sigmoid(z);
        
        J = (
                (1/self.m) 
                * np.sum( 
                            np.multiply(
                                (-1*self.y),
                                np.log(h)
                            )
                            - 
                            np.multiply(
                                np.subtract(1, self.y), 
                                np.log(1-h)
                            )
                )  
            ) + \
            ( 
                (self.lam/self.m) 
                * sum(
                    np.square(
                        np.vstack(
                            [0, theta[1:]]
                        )
                    )
                ) 
            );
        return J;
    def gradFunction(self, theta):
        
#         print (type(theta), theta.shape, theta);
        
        theta = np.reshape(theta,(self.n + 1, 1));
        
        z = self.X * theta;
        h = sigmoid(z);
        
#         print(
        
#             "h shape", h.shape,
#             "self.y shape", self.y.shape,
# #             "y", y,
#             "h-self.y shape", (h-self.y).shape,
#             "self.X shape", (self.X).shape,
#             "self.X t shape", np.transpose(self.X).shape,
            
#             "needed", np.transpose(self.X) * (h-self.y),
            
#             self.X,
#             theta,
#             self.y
        
#         );
        
        grad = ( 
                    (1/self.m) 
                    * 
                    (
                        np.transpose(self.X)
                        * (h - self.y)
                    ) 
                ) + \
                ( 
                    (self.lam/self.m)
                    * 
                    np.vstack(
                        [0, theta[1:]]
                    ) 
                );
#         print(
#             grad,
#             np.asarray(grad).reshape((3,)),
#             type(np.asarray(grad))
#         )
        return np.asarray(grad).reshape((self.n + 1,));
sampleX = np.matrix('1 2; 3 4');
sampleY = np.matrix('1;1')
lgTest = LogisticRegression(sampleX, sampleY, 1);
opTheta, fminOutput = lgTest.buildModel();
LogisticRegression.sigmoid(LogisticRegression.addIntercept(sampleX) * opTheta)


        

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 29
         Function evaluations: 32
         Gradient evaluations: 32


matrix([[ 0.99999998],
        [ 0.99999998]])

In [235]:
X = np.matrix('1 1 2;1 3 4')
y = np.matrix('1;1')
theta = np.matrix('1;2;3')
a= np.subtract(1,y)
b = np.log(X*theta)
np.multiply(np.subtract(1,y),np.log(X*theta))

matrix([[ 0.],
        [ 0.]])

In [161]:
def A(x):
    return x + 3;
def B(x):
    return x + 6;
def C(x):
    return x + 9;
print(
    A(B(C(0)))
);

18


In [111]:
A = np.matrix('1 2 3; 3 4 3');
def test(A):
    return [A, A.shape[0]];
b = np.matrix('1;2;3')
c= np.matrix([123])
c
b
np.vstack([0, b[1:]])
np.subtract(1, b)
1/ (1 +np.exp(-1 * b))
# print ( 1, 'test')
# b[1:]


matrix([[ 0.73105858],
        [ 0.88079708],
        [ 0.95257413]])

In [185]:
class test:
    def costFunction(self, x):
        return x-5;

print(fmin_bfgs(test().costFunction, x0 = 1))

(-3.57913940e+08) -5


Optimization terminated successfully.
         Current function value: -357913945.000000
         Iterations: 1
         Function evaluations: 48
         Gradient evaluations: 16
[ -3.57913940e+08]


-357913945.0